# Generamos Fotos a partir de los datos históricos
Importamos las librerias necesarias

In [1]:
import os
import pandas as pd

import tfm_helpers as tfm

pd.set_option("display.max_columns", None)

_Definiciones de ficheros_
* Hoteles

In [3]:
hoteles = ["02", "04", "05", "15", "16", "21", "22"]

data_source_path = r"C:\Datos\UOC\OneDrive - Universitat Oberta de Catalunya\TFM\Datos"


* Reservas

In [4]:
reservas_dtypes = {
    "Hotel" : str,
    "Cliente" : str,
    "TTOO" : str,
    "Agencia" : str,
    "Noches" : int,
    "Cantidad habitaciones" : int,
    "Nacionalidad" : str,
    "Régimen factura" : str,
    "Tipo facturación estancia" : str,
    "Estado reserva" : str,
    "Desglosada" : int,
    "Usuario venta" : str,
    "Usuario Borrado" : str,
    "Day Use" : int,
    "Grupo" : str,
    "Master del grupo" : int,
    "Canal_Origen" : str,
    "Segmento" : str,
    "Régimen uso" : str,
    "Canal_Origen Aux" : str,
    "Id Reserva" : str,
    "Id Tipo habitacion uso" : str,
    "Id Tipo habitacion factura" : str
}

reservas_parse_dates = [
    "Fecha venta", 
    "Fecha entrada", 
    "Fecha salida",
    "Fecha Borrado"
]

reservas_cols = list(reservas_dtypes.keys()) + reservas_parse_dates + ["AD", "JR", "NI", "CU"]



* Reservas Diarias

In [5]:
reservas_diarias_dtypes = {
    "Hotel" : str,
    "Régimen factura" : str,
    "Estado reserva" : str,
    "Régimen uso" : str,
    "Day Use" : int,
    "Canal_Origen" : str,
    "Segmento" : str,
    "Id Reserva" : str,
    "Id Tipo habitacion uso" : str,
    "Id Tipo habitacion factura" : str,
    "Id Habitacion" : str
}

reservas_diarias_parse_dates = ["Fecha"]

reservas_diarias_cols = list(reservas_diarias_dtypes.keys()) + reservas_diarias_parse_dates + ["Ocupación", "AD", "JR", "NI", "CU"]

Creamos funciones de uso genérico a lo largo del script

El proceso sigue el siguiente algoritmo:
* Cargar todas las Reservas de un hotel en un DataSet
* Empezar a generar fotos para cada día desde la primera fecha de venta que tengamos
	* Para cada día desde MinFechaVenta hasta MaxFechaVenta
		* Filtrar Para cada Fecha en Lista
			* FechaVenta <= Fecha y FechaBorrado >= Fecha y Estado En Reserva, Entrada, Salida
				* Generar un fichero para ese día con todos los datos
				
_Unificamos las Reservas_

In [21]:
for hotel in hoteles:   
    files = tfm.getListOfFiles(tfm.getSourcePathToHistoricalReservasHotel(data_source_path, hotel))

    all_years_reservas_hotel = pd.DataFrame()
    for file in files:   
        current_data = pd.read_csv(
                                    file, 
                                    sep="|", 
                                    escapechar="~",
                                    encoding="cp1252", 
                                    on_bad_lines="skip", 
                                    encoding_errors="ignore",
                                    quoting=3,
                                    usecols=reservas_cols,
                                    dtype=reservas_dtypes, 
                                    parse_dates=reservas_parse_dates,
                                    converters={"AD":tfm.to_integer, "JR":tfm.to_integer, "NI":tfm.to_integer, "CU":tfm.to_integer}                                
                                )
        all_years_reservas_hotel = pd.concat([all_years_reservas_hotel, current_data], ignore_index=True)
    
    all_years_reservas_hotel.to_csv(tfm.getPathToProcessedHistoricalReservasHotel(hotel, "csv"), index=False, sep="|", escapechar="~", encoding="cp1252")
    all_years_reservas_hotel.to_parquet(tfm.getPathToProcessedHistoricalReservasHotel(hotel, "parquet"), index=False)

_Unificamos las Reservas Diarias_

In [6]:
for hotel in hoteles: 
    files = tfm.getListOfFiles(tfm.getSourcePathToHistoricalReservasDiariasHotel(data_source_path, hotel))

    all_years_reservas_diarias = pd.DataFrame()
    for file in files:
        current_data = pd.read_csv(
                                    file, 
                                    sep="|", 
                                    escapechar="~",
                                    encoding="cp1252", 
                                    on_bad_lines="skip", 
                                    encoding_errors="ignore",
                                    quoting=3,
                                    usecols=reservas_diarias_cols,
                                    dtype=reservas_diarias_dtypes, 
                                    parse_dates=reservas_diarias_parse_dates, 
                                    converters={"Ocupación":tfm.to_integer, "AD":tfm.to_integer, "JR":tfm.to_integer, "NI":tfm.to_integer, "CU":tfm.to_integer} 
                                )
        all_years_reservas_diarias = pd.concat([all_years_reservas_diarias, current_data], ignore_index=True)
        
    all_years_reservas_diarias.to_csv(tfm.getPathToProcessedHistoricalReservasDiariasHotel(hotel, "csv"), index=False, sep="|", escapechar="~", encoding="cp1252")
    all_years_reservas_diarias.to_parquet(tfm.getPathToProcessedHistoricalReservasDiariasHotel(hotel, "parquet"), index=False)